In [1]:
import os 
%pwd


'd:\\My Workspace\\proj_file\\End-To-End-Wine-Quality-Pred\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\My Workspace\\proj_file\\End-To-End-Wine-Quality-Pred'

In [5]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    target_column: str

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        
        config = self.config.model_trainer
        params = self.params.TUNING
        schema = self.schema.TARGET_COLUMN
        
        
        create_directories([config.root_dir])
        
        
        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            n_estimators= params.n_estimators,
            target_column= schema.name,
            
        )
        
        return model_trainer_config

In [8]:
import pandas as pd
import os
import logging
import joblib


from sklearn.ensemble import RandomForestClassifier



class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        x_train = train_data.drop([self.config.target_column], axis=1)
        x_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        model = RandomForestClassifier(n_estimators=self.config.n_estimators)
        model.fit(x_train, y_train)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))


In [9]:
from src.exception import CustomException
import sys

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise CustomException(e,sys)


[ 2024-06-22 09:09:54,292 ] 33 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2024-06-22 09:09:54,294 ] 33 root - INFO - yaml file: params.yaml loaded successfully
[ 2024-06-22 09:09:54,297 ] 33 root - INFO - yaml file: schema.yaml loaded successfully
[ 2024-06-22 09:09:54,299 ] 53 root - INFO - created directory at: artifacts
[ 2024-06-22 09:09:54,300 ] 53 root - INFO - created directory at: artifacts/model_trainer


c:\Users\VIYAS\anaconda3\envs\ETEwine\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
